Reproduce the tutorial analysis with data provided by basenji tutorial

The input you bring to the pipeline is:
* BigWig coverage tracks
* Genome FASTA file

They provide a machine learning friendly simplified version of hg19 FASTA file;

BigWig data was a few CAGE datasets from FANTOM5 related to heart biology and processed by:
1. Aligning with Bowtie2 with very sensitive alignment parameters.
2. Distributing multi-mapping reads and estimating genomic coverage with [bam_cov.py](https://github.com/calico/basenji/blob/master/bin/bam_cov.py)

In [1]:
# confirm the download directory
import os

print(f"before work dir: {os.getcwd()}")
# change to parent directory
os.chdir("..")
print(f"after work dir: {os.getcwd()}")

before work dir: /Users/library/Documents/ML_project/basenji_genomics_ai/experiments/reproduction/notebooks
after work dir: /Users/library/Documents/ML_project/basenji_genomics_ai/experiments/reproduction


In [ ]:
# download fasta files
import subprocess
if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   729  100   729    0     0    893      0 --:--:-- --:--:-- --:--:--   916


In [ ]:
# download BigWig files
if not os.path.isfile('data/CNhs11760.bw'):
    subprocess.call('curl -o data/CNhs11760.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs11760.bw', shell=True)
    subprocess.call('curl -o data/CNhs12843.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12843.bw', shell=True)
    subprocess.call('curl -o data/CNhs12856.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12856.bw', shell=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1563M  100 1563M    0     0  16.8M      0  0:01:32  0:01:32 --:--:-- 18.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1236M  100 1236M    0     0  17.5M      0  0:01:10  0:01:10 --:--:-- 19.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3484M  100 3484M    0     0  11.3M      0  0:05:06  0:05:06 --:--:-- 7945k


In [7]:
lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'CNhs11760', 'data/CNhs11760.bw', '384', 'sum', 'aorta'])
lines.append(['1', 'CNhs12843', 'data/CNhs12843.bw', '384', 'sum', 'artery'])
lines.append(['2', 'CNhs12856', 'data/CNhs12856.bw', '384', 'sum', 'pulmonic_valve'])

samples_out = open('data/heart_wigs.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

Next, we want to choose genomic sequences to form batches for stochastic gradient descent, divide them into training/validation/test sets, and construct TFRecords to provide to downstream programs.

The script [basenji_data.py](https://github.com/calico/basenji/blob/master/bin/basenji_data.py) implements this procedure.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -s | 0.1 | Down-sample the genome to 10% to speed things up here. |
| -g | data/unmap_macro.bed | Dodge large-scale unmappable regions like assembly gaps. |
| -l | 131072 | Sequence length. |
| --local | True | Run locally, as opposed to on my SLURM scheduler. |
| -o | data/heart_l131k | Output directory |
| -p | 8 | Uses multiple concourrent processes to read/write. |
| -t | .1 | Hold out 10% sequences for testing. |
| -v | .1 | Hold out 10% sequences for validation. |
| -w | 128 | Pool the nucleotide-resolution values to 128 bp bins. |
| fasta_file| data/hg19.ml.fa | FASTA file to extract sequences from. |
| targets_file | data/heart_wigs.txt | Target samples table with BigWig paths. |

In [8]:
# process the data into a dataset
! python basenji_data.py -s .1 -g data/unmap_macro.bed -l 131072 --local -o data/heart_l131k -p 8 -t .1 -v .1 -w 128 data/hg19.ml.fa data/heart_wigs.txt

stride_train 1 converted to 131072.000000
stride_test 1 converted to 131072.000000
Contigs divided into
 Train:  4701 contigs, 2169074921 nt (0.8005)
 Valid:   572 contigs,  270358978 nt (0.0998)
 Test:    584 contigs,  270330829 nt (0.0998)
python basenji_data_read.py -w 128 -u sum -c 384.000000 -s 1.000000 data/CNhs11760.bw data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov/0.h5
python basenji_data_read.py -w 128 -u sum -c 384.000000 -s 1.000000 data/CNhs12843.bw data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov/1.h5
python basenji_data_read.py -w 128 -u sum -c 384.000000 -s 1.000000 data/CNhs12856.bw data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov/2.h5
Targets sum: 160677.829
Targets sum: 276909.169
Targets sum: 743743.941
python basenji_data_write.py -s 0 -e 256 --umap_clip 1.000000 -x 0 data/hg19.ml.fa data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov data/heart_l131k/tfrecords/train-0.tfr
python basenji_data_write.py -s 256 -e 512 --umap_clip 1.000000

Now, data/heart_l131k contains relevant data for training.

*contigs.bed* contains the original large contiguous regions from which training sequences were taken (possibly strided).

*sequences.bed* contains the train/valid/test sequences.

In [9]:
! cut -f4 data/heart_l131k/sequences.bed | sort | uniq -c

 179 test
1499 train
 180 valid


In [10]:
! head -n3 data/heart_l131k/sequences.bed

chr2	140425791	140556863	train
chr16	27143973	27275045	train
chr14	72972403	73103475	train


In [11]:
! grep valid data/heart_l131k/sequences.bed | head -n3

chr11	116271608	116402680	valid
chr2	26239628	26370700	valid
chr6	102241368	102372440	valid


In [12]:
! grep test data/heart_l131k/sequences.bed | head -n3

chr14	76007464	76138536	test
chr13	99391330	99522402	test
chr15	52280212	52411284	test


In [13]:
! ls -l data/heart_l131k/tfrecords/*.tfr

-rw-r--r--  1 library  staff   7205322 Sep 28 15:45 data/heart_l131k/tfrecords/test-0.tfr
-rw-r--r--  1 library  staff  10314025 Sep 28 15:45 data/heart_l131k/tfrecords/train-0.tfr
-rw-r--r--  1 library  staff  10276124 Sep 28 15:45 data/heart_l131k/tfrecords/train-1.tfr
-rw-r--r--  1 library  staff  10317142 Sep 28 15:45 data/heart_l131k/tfrecords/train-2.tfr
-rw-r--r--  1 library  staff  10318764 Sep 28 15:45 data/heart_l131k/tfrecords/train-3.tfr
-rw-r--r--  1 library  staff  10302540 Sep 28 15:45 data/heart_l131k/tfrecords/train-4.tfr
-rw-r--r--  1 library  staff   8828267 Sep 28 15:45 data/heart_l131k/tfrecords/train-5.tfr
-rw-r--r--  1 library  staff   7244099 Sep 28 15:45 data/heart_l131k/tfrecords/valid-0.tfr
